In [8]:
# convert_to_webp.py
from __future__ import annotations
import os
from pathlib import Path
from typing import Iterable, Tuple, Optional
from PIL import Image, ImageOps, UnidentifiedImageError

# Optional decoders (import if installed)
try:
    from pillow_heif import register_heif_opener  # HEIC/HEIF
    register_heif_opener()
except Exception:
    pass

try:
    import pillow_avif_plugin  # noqa: F401  # AVIF
except Exception:
    pass


DEFAULT_EXTS = {
    ".jpg", ".jpeg", ".png", ".gif", ".bmp",
    ".tif", ".tiff",
    ".heic", ".heif", ".heics", ".heifs",
    ".avif"
}

def convert_any_to_webp(
    src: Path,
    dst: Path,
    *,
    quality: int = 80,
    method: int = 6,
    max_dim: Optional[int] = None,     # e.g., 2560 to downscale large images
    strip_metadata: bool = True,       # drop EXIF to reduce size
    recompress_even_if_webp: bool = False,  # allow recompressing .webp inputs
) -> Tuple[bool, str]:
    """
    Convert image at `src` to WebP at `dst`. Returns (success, message).
    Keeps animation for GIF/animated inputs. Preserves alpha if present.
    """
    try:
        with Image.open(src) as im:
            # Auto-rotate according to EXIF
            try:
                im = ImageOps.exif_transpose(im)
            except Exception:
                pass

            # Optional downscale (preserves aspect ratio)
            if max_dim and all(x is not None for x in im.size):
                w, h = im.size
                scale = min(max_dim / w, max_dim / h, 1.0)
                if scale < 1.0:
                    new_size = (max(1, int(w * scale)), max(1, int(h * scale)))
                    im = im.resize(new_size, Image.LANCZOS)

            # Mode conversion to keep alpha when present
            if "A" in im.getbands():   # has alpha
                im = im.convert("RGBA")
            elif im.mode not in ("RGB", "RGBA"):
                im = im.convert("RGB")

            info = im.info.copy()

            # Animated? (GIF or animated WebP/others)
            is_animated = bool(getattr(im, "is_animated", False))
            save_kwargs = dict(
                format="WEBP",
                quality=quality,
                method=method,
                lossless=False,     # set True for PNG-like visuals, but bigger
                optimize=True,
            )

            # Metadata handling
            if not strip_metadata:
                # keep ICC if available; EXIF kept automatically if present
                icc = info.get("icc_profile")
                if icc:
                    save_kwargs["icc_profile"] = icc
            else:
                # Strip EXIF/metadata
                info.pop("exif", None)
                info.pop("icc_profile", None)

            # Animated saves
            if is_animated:
                save_kwargs.update(
                    save_all=True,
                    duration=info.get("duration"),
                    loop=info.get("loop", 0),
                )

            dst.parent.mkdir(parents=True, exist_ok=True)
            im.save(dst, **save_kwargs)

        return True, f"✅ {src} -> {dst}"
    except UnidentifiedImageError as e:
        return False, f"❌ Unrecognized image: {src} ({e})"
    except Exception as e:
        return False, f"❌ Failed: {src}\n   {e}"


def convert_folder_to_webp(
    root_folder: os.PathLike | str,
    *,
    include_exts: Iterable[str] = DEFAULT_EXTS,
    inplace: bool = True,              # True: same tree; False: write under webp_output/
    overwrite: bool = False,           # skip if dst exists unless overwrite=True
    recompress_existing_webp: bool = False,
    quality: int = 80,
    method: int = 6,
    max_dim: Optional[int] = None,
    strip_metadata: bool = True,
    delete_original_nonwebp: bool = False,
) -> None:
    root = Path(root_folder).expanduser().resolve()
    include_exts = {e.lower() for e in include_exts}

    total = converted = skipped = failed = 0

    for src in root.rglob("*"):
        if not src.is_file():
            continue
        ext = src.suffix.lower()

        # Decide whether to process this file:
        if ext == ".webp":
            if not recompress_existing_webp:
                # Leave existing .webp alone
                continue
            # We will recompress .webp -> .webp
        elif ext not in include_exts:
            # Not a target format
            continue

        total += 1

        dst = (
            src.with_suffix(".webp")
            if inplace
            else (root / "webp_output" / src.relative_to(root)).with_suffix(".webp")
        )

        if dst.exists() and not overwrite:
            print(f"⏭️  Skip (exists): {dst}")
            skipped += 1
            continue

        ok, msg = convert_any_to_webp(
            src, dst,
            quality=quality,
            method=method,
            max_dim=max_dim,
            strip_metadata=strip_metadata,
            recompress_even_if_webp=recompress_existing_webp,
        )
        print(msg)
        if ok:
            converted += 1
            # Remove original if requested and we converted a non-webp → webp
            if delete_original_nonwebp and src.suffix.lower() != ".webp":
                try:
                    src.unlink()
                    print(f"🗑️  Deleted original: {src}")
                except Exception as e:
                    print(f"⚠️  Couldn't delete original {src}: {e}")
        else:
            failed += 1

    print("\n—— Summary ——")
    print(f"Visited:     {total}")
    print(f"Converted:   {converted}")
    print(f"Skipped:     {skipped}")
    print(f"Failed:      {failed}")


if __name__ == "__main__":
    convert_folder_to_webp(
        "images/",
        inplace=True,
        overwrite=False,                # keep False unless you want to overwrite existing .webp
        recompress_existing_webp=False, # only convert non-webp inputs
        quality=80,
        method=6,
        max_dim=None,                   # e.g., 1920 or 2560 to downscale large files
        strip_metadata=True,
        delete_original_nonwebp=True,   # <-- delete originals after successful conversion
    )

    # 2) (Optional) Recompress existing webp as well, and downscale to 2560px max
    # convert_folder_to_webp(
    #     "images/info/about/",
    #     inplace=True,
    #     overwrite=True,
    #     recompress_existing_webp=True,
    #     quality=78,
    #     method=6,
    #     max_dim=2560,
    #     strip_metadata=True,
    #     delete_original_nonwebp=False,
    # )


✅ /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_7852.HEIC -> /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_7852.webp
🗑️  Deleted original: /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_7852.HEIC
✅ /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_8709.HEIC -> /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_8709.webp
🗑️  Deleted original: /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_8709.HEIC
✅ /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_1683.HEIC -> /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_1683.webp
🗑️  Deleted original: /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_1683.HEIC
✅ /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_9476.HEIC -> /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_9476.webp
🗑️  Deleted original: /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_9476.HEIC
✅ /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_8380.HEIC -> /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_8380.webp
🗑️  Deleted original: /Users/77wu/Documents/GitHub/Blog2.0/images/IMG_8380.

In [2]:
convert_heic_to_webp(
    "images/",
    quality=80,
    method=6,
    inplace=True,        # write .webp next to the source
    overwrite=False,     # keep existing .webp if present
    delete_original=True # 👈 remove the old HEICs after convert
)



—— Summary ——
Found HEICs: 0
Converted:   0
Skipped:     0
Failed:      0
